# Audio Acquisition Test
A short example on how to use the audio acqusition tools.
The first cell loads the required packages, the second cell controlls the logging outputs.

In [1]:
import numpy as np
from numpy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from time import sleep
import scipy.signal as signal
from queue import Queue
import logging

import sys
sys.path.insert(0,'..')

import audio_acquisition.handlers
import audio_acquisition.ni
import audio_acquisition.triggers
import audio_acquisition.utils

In [2]:
audio_acquisition.handlers.logger.setLevel(logging.INFO)
audio_acquisition.triggers.logger.setLevel(logging.INFO)
audio_acquisition.logger.addHandler(logging.StreamHandler(sys.stdout))

## Initialization
This cell has to be run once for each measurement. In the current implementation it is not possible to reuse a setup.
This is because threads cannot be restarted once finished. If it is necessary for a system to be started and stopped repeatedly it is better to use some kind of intelligent triggering system to pause the acquisition.

In [3]:
device = audio_acquisition.ni.NIDevice()
device.add_input(0)
device.blocksize = 10000
handler = audio_acquisition.handlers.DeviceHandler(device)
trigger = audio_acquisition.triggers.RMSTrigger(level=1, channel=0, fs=device.fs, 
                                                action=handler.trigger_handler.trigger.set)
handler.trigger_handler.triggers.append(trigger)
# handler.trigger_handler.trigger.set()
Q = Queue()
handler.queue_handler.queues.append(Q)

## Running
In order to start the system, execute `handler.start()`. use triggers to control when the actual measurement starts. Execute `handler.stop()` to finalize the measurement and let all threads finish.